In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import *
from utils import load_input

# Part 1 

Unfortunately, the ship's navigation computer seems to be malfunctioning; rather than giving a route directly to safety, it produced extremely circuitous instructions. When the captain uses the PA system to ask if anyone can help, you quickly volunteer.

The navigation instructions (your puzzle input) consists of a sequence of single-character actions paired with integer input values. After staring at them for a few minutes, you work out what they probably mean:
```
Action N means to move north by the given value.
Action S means to move south by the given value.
Action E means to move east by the given value.
Action W means to move west by the given value.
Action L means to turn left the given number of degrees.
Action R means to turn right the given number of degrees.
Action F means to move forward by the given value in the direction the ship is currently facing.
```
The ship starts by facing east. Only the L and R actions change the direction the ship is facing. (That is, if the ship is facing east and the next instruction is N10, the ship would move north 10 units, but would still move east if the following action were F.)

For example:

F10
N3
F7
R90
F11
These instructions would be handled as follows:

F10 would move the ship 10 units east (because the ship starts by facing east) to east 10, north 0.
N3 would move the ship 3 units north to east 10, north 3.
F7 would move the ship another 7 units east (because the ship is still facing east) to east 17, north 3.
R90 would cause the ship to turn right by 90 degrees and face south; it remains at east 17, north 3.
F11 would move the ship 11 units south to east 17, south 8.
At the end of these instructions, the ship's Manhattan distance (sum of the absolute values of its east/west position and its north/south position) from its starting position is 17 + 8 = 25.

Figure out where the navigation instructions lead. What is the Manhattan distance between that location and the ship's starting position?



In [4]:
data = load_input(12)

In [6]:
instructions = [(d[0], int(d[1:])) for d in data]

In [18]:
from collections import namedtuple
Point = namedtuple("Point", "x y")
current_pos = Point(0, 0)
current_dir = "E"

In [19]:
left_turn = {
    'E': 'N',
    'N': 'W',
    'W': 'S',
    'S': 'E'
}

right_turn = {
    'E': 'S',
    'S': 'W',
    'W': 'N',
    'N': 'E'
}

def turn(current, turn_dir, degrees):
    state_dict = left_turn if turn_dir == "L" else right_turn
    out = current
    for _ in range(degrees//90):
        out = state_dict[out]
    return out

state_transition = {
    'N': lambda p, d, u: (Point(p.x, p.y+u), d),
    'S': lambda p, d, u: (Point(p.x, p.y-u), d),
    'E': lambda p, d, u: (Point(p.x+u, p.y), d),
    'W': lambda p, d, u: (Point(p.x-u, p.y), d),
    'L': lambda p, d, u: (p, turn(d, 'L', u)),
    'R': lambda p, d, u: (p, turn(d, 'R', u)),
    'F': lambda p, d, u: state_transition[d](p, d, u)
}    
    

In [20]:
for action, unit in instructions:
    f = state_transition[action]
    current_pos, current_dir = f(current_pos, current_dir, unit)

In [22]:
abs(current_pos.x)+abs(current_pos.y)

1441

# Part 2

Before you can give the destination to the captain, you realize that the actual action meanings were printed on the back of the instructions the whole time.

Almost all of the actions indicate how to move a waypoint which is relative to the ship's position:
```
Action N means to move the waypoint north by the given value.
Action S means to move the waypoint south by the given value.
Action E means to move the waypoint east by the given value.
Action W means to move the waypoint west by the given value.
Action L means to rotate the waypoint around the ship left (counter-clockwise) the given number of degrees.
Action R means to rotate the waypoint around the ship right (clockwise) the given number of degrees.
Action F means to move forward to the waypoint a number of times equal to the given value.
```

The waypoint starts 10 units east and 1 unit north relative to the ship. The waypoint is relative to the ship; that is, if the ship moves, the waypoint moves with it.

For example, using the same instructions as above:
```
F10 moves the ship to the waypoint 10 times (a total of 100 units east and 10 units north), leaving the ship at east 100, north 10. The waypoint stays 10 units east and 1 unit north of the ship.
N3 moves the waypoint 3 units north to 10 units east and 4 units north of the ship. The ship remains at east 100, north 10.
F7 moves the ship to the waypoint 7 times (a total of 70 units east and 28 units north), leaving the ship at east 170, north 38. The waypoint stays 10 units east and 4 units north of the ship.
R90 rotates the waypoint around the ship clockwise 90 degrees, moving it to 4 units east and 10 units south of the ship. The ship remains at east 170, north 38.
F11 moves the ship to the waypoint 11 times (a total of 44 units east and 110 units south), leaving the ship at east 214, south 72. The waypoint stays 4 units east and 10 units south of the ship.
```
After these operations, the ship's Manhattan distance from its starting position is 214 + 72 = 286.

Figure out where the navigation instructions actually lead. What is the Manhattan distance between that location and the ship's starting position?



In [71]:
class Ship:
    
    rotation = {
        'L': lambda w: Point(-w.y, w.x),
        'R': lambda w: Point(w.y, -w.x)
    }
    
    state_transition = {
        'N': lambda p, w, v: (p, Point(w.x, w.y+v)),
        'S': lambda p, w, v: (p, Point(w.x, w.y-v)),
        'E': lambda p, w, v: (p, Point(w.x+v, w.y)),
        'W': lambda p, w, v: (p, Point(w.x-v, w.y)),
        'L': lambda p, w, v: (p, Ship.rotate_waypoint(w, 'L', v)),
        'R': lambda p, w, v: (p, Ship.rotate_waypoint(w, 'R', v)),
        'F': lambda p, w, v: (Point(p.x+(v*w.x),p.y+(v*w.y)) , w)
    }    
    
    def __init__(self):
        self.pos = Point(0, 0)
        self.w_pos = Point(10, 1)
        
        
    def move(self, action, value):
        f = self.state_transition[action]
        self.pos, self.w_pos = f(self.pos, self.w_pos, value)
    
    @classmethod
    def rotate_waypoint(cls, waypoint_loc, turn_dir, value):
        #new_w_loc = waypoint_loc
        for _ in range(value//90):
            waypoint_loc = cls.rotation[turn_dir](waypoint_loc)

        return waypoint_loc
        
    

In [64]:
ship = Ship()

In [65]:
for action, value in instructions:
    ship.move(action, value)

In [66]:
abs(ship.pos.x)+abs(ship.pos.y)

61616